<a href="https://colab.research.google.com/github/Surya-Abhinai/object-detection-/blob/main/Boundary_box.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import division
import torch 
import random 
import numpy as np
import cv2 

In [2]:
def confidence_filter(result, confidence):
    # result is a 3D tensor.The first four elements in the last tensor represent x_cor , y_cor of boundary boxes, and it's width and height.
    #The fifth element represents the confidence level of the prediction of the boundary box.
    confidence_boolean_mask = (result[:,:,4] > confidence).float().unsqueeze(2)

    # multiplying result with confidence_boolean_mask tensor.
    result = result*confidence_boolean_mask    

    return result

In [4]:
def confidence_filter_class(result, confidence):
    #result tensor shape = (batch_size,num_of_anchors,num_classes + 5)

    # extract maximum scores for each class and concatenate them with result .
    max_scores = torch.max(result[:,:,5:], 2)[0]
    res = torch.cat((result, max_scores),2)
    
    #object confidence score greater than the specified confidence threshold
    cond_1 = (res[:,:,4] > confidence).float()
    #class-specific confidence score greater than 0.995
    cond_2 = (res[:,:,-1] > 0.995).float()
    
    # combine the two conditions using element-wise addition and clamp the values between 0 and 1
    conf = cond_1 + cond_2
    conf = torch.clamp(conf, 0.0, 1.0)

    # add an extra dimension to the confidence tensor and apply it to the result tensor to filter out low-confidence objects
    conf = conf.unsqueeze(2)
    result = result*conf   
    return result

In [28]:
def get_abs_coord(box):
    # box = (centre_x_cor , centre_y_cor , width , height)
    box[2], box[3] = abs(box[2]), abs(box[3])

    # top_left coordinates (x1,y1)
    x1 = (box[0] - box[2]/2) - 1
    y1 = (box[1] - box[3]/2)  - 1
    # bottom_right coordinates (x2,y2)
    x2 = (box[0] + box[2]/2)  - 1
    y2 = (box[1] + box[3]/2) - 1
    return x1, y1, x2, y2

In [25]:
def sanity_fix(box):
    # If x1 > x2 or y1 > y2, it swaps the values so that they are in the correct order.
    if (box[0] > box[2]):
        box[0], box[2] = box[2], box[0]
    
    if (box[1] >  box[3]):
        box[1], box[3] = box[3], box[1]
        
    return box

In [27]:
def bbox_iou(box1, box2):

    #Get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1[:,0], box1[:,1], box1[:,2], box1[:,3]
    b2_x1, b2_y1, b2_x2, b2_y2 = box2[:,0], box2[:,1], box2[:,2], box2[:,3]
    
    #get the corrdinates of the intersection rectangle
    inter_rect_x1 =  torch.max(b1_x1, b2_x1)
    inter_rect_y1 =  torch.max(b1_y1, b2_y1)
    inter_rect_x2 =  torch.min(b1_x2, b2_x2)
    inter_rect_y2 =  torch.min(b1_y2, b2_y2)
    
    #Intersection area
    inter_area = torch.max(inter_rect_x2 - inter_rect_x1 + 1,torch.zeros(inter_rect_x2.shape))*torch.max(inter_rect_y2 - inter_rect_y1 + 1, torch.zeros(inter_rect_x2.shape))
    
    #Union Area
    b1_area = (b1_x2 - b1_x1 + 1)*(b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1)*(b2_y2 - b2_y1 + 1)
    
    iou = inter_area / (b1_area + b2_area - inter_area)
    
    return iou


In [29]:
def pred_corner_coord(prediction):
    # Get indices of non-zero confidence bboxes
    ind_nz = torch.nonzero(prediction[:,:,4]).transpose(0,1).contiguous()
    # Extract the boxes with non-zero confidence
    box = prediction[ind_nz[0], ind_nz[1]]

    # Convert the center-coordinate format to corner-coordinate format
    box_a = box.new(box.shape)
    box_a[:,0] = (box[:,0] - box[:,2]/2)   # x1 = xc - w/2
    box_a[:,1] = (box[:,1] - box[:,3]/2)   # y1 = yc - h/2
    box_a[:,2] = (box[:,0] + box[:,2]/2)   # x2 = xc + w/2
    box_a[:,3] = (box[:,1] + box[:,3]/2)   # y2 = yc + h/2
    box[:,:4] = box_a[:,:4]

    # Replace the center-coordinate format with the corner-coordinate format
    prediction[ind_nz[0], ind_nz[1]] = box

    return prediction


In [31]:
def write(x, batches, results, colors, classes):
    # x[0]: the index of the image in the batch to which the detected object belongs
    # x[1]: the x-coordinate of the top-left corner of the bounding box around the object
    # x[2]: the y-coordinate of the top-left corner of the bounding box around the object
    # x[3]: the x-coordinate of the bottom-right corner of the bounding box around the object
    # x[4]: the y-coordinate of the bottom-right corner of the bounding box around the object
    # x[5]: the index of the class label of the object
    c1 = tuple(x[1:3].int())
    c2 = tuple(x[3:5].int())
    img = results[int(x[0])]
    cls = int(x[-1])
    label = "{0}".format(classes[cls])
    color = random.choice(colors)
    cv2.rectangle(img, c1, c2,color, 1)
    t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
    c2 = c1[0] + t_size[0] + 3, c1[1] + t_size[1] + 4
    cv2.rectangle(img, c1, c2,color, -1)
    cv2.putText(img, label, (c1[0], c1[1] + t_size[1] + 4), cv2.FONT_HERSHEY_PLAIN, 1, [225,255,255], 1);
    return img